In [ ]:
import threading
import json
import time
from kafka import KafkaProducer, KafkaConsumer

In [ ]:
class Counter:
    def __init__(self):
        self._lock = threading.Lock()
        self.reset()
        
    def reset(self):
        with self._lock:
            self.counter = 0
            
    def incr(self, value):
        with self._lock:
            self.counter += value
            
    def get(self):
        return self.counter

In [ ]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'localhost:9092'
data_topic = 'data'
notification_topic = 'notification'

In [ ]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [ ]:
consumer = KafkaConsumer(
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: x.decode('utf-8'))
consumer.subscribe(data_topic)

In [ ]:
def monitor_thread(interval, counters, topic):
    print('[monitor] starting')
    while True:
        time.sleep(interval)
        for id in counters:
            count = counters[id].get()
            counters[id].reset()
            s = '{} - {} messages during last {} seconds'.format(id, count, interval)
            producer.send(topic, s.encode('utf-8'))
        print('send notificiation', flush=True)

In [ ]:
monitor_interval = 10
counters = {}
monitor = threading.Thread(target=monitor_thread, args=(monitor_interval, counters, notification_topic), daemon=True)
monitor.start()

In [ ]:
for message in consumer:
    m = message.value
    data = json.loads(m)
    if 'id' in data:
        if data['id'] not in counters:
            counters[data['id']] = Counter()
        counters[data['id']].incr(1)
    else:
        print(data, flush=True)